# MIMIC-IV query API tutorial

This notebook shows examples of how to use the cyclops.query API on [MIMIC-IV v2.0](https://physionet.org/content/mimiciv/2.0/).

* First, setup the MIMIC-IV database according to the instructions in [mimic-code](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `MIMICIVQuerier` accordingly.

## Imports and instantiate `MIMICIVQuerier`

In [1]:
import cyclops.query.ops as qo
from cyclops.query import MIMICIVQuerier

mimic = MIMICIVQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciv-2.0",
    user="postgres",
    password="pwd",
)
# List all tables.
mimic.list_tables()

2023-01-30 15:16:11,459 INFO cyclops.query.orm - Database setup, ready to run queries!


['patients',
 'admissions',
 'diagnoses',
 'dim_diagnoses',
 'dim_labitems',
 'dim_items',
 'chartevents',
 'labevents',
 'pharmacy',
 'transfers',
 'ed_stays']

## Example 1. Get all patient admissions from 2021 or later (approx year of admission)

In [2]:
patients = mimic.patients()
ops = qo.Sequential(
    [
        qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
        qo.ConditionAfterDate("admittime", "2021-01-01"),
    ]
)
admissions = mimic.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
    ops=ops,
).run()
print(f"{len(admissions)} rows extracted!")

2023-01-30 15:16:11,879 INFO cyclops.query.orm - Query returned successfully!
2023-01-30 15:16:11,880 INFO cyclops.utils.profile - Finished executing function run_query in 0.284805 s


1575 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD-10 long title), in the year 2015.

In [3]:
diagnoses_ops = qo.Sequential(
    [
        qo.ConditionEquals("icd_version", 10),
        qo.ConditionSubstring("long_title", "schizophrenia"),
    ]
)
admissions_ops = qo.Sequential(
    [
        qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
        qo.ConditionInYears("admittime", "2015"),
    ]
)
patients = mimic.patients()
admissions = mimic.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
    ops=admissions_ops,
)
diagnoses = mimic.diagnoses(
    join=qo.JoinArgs(
        join_table=admissions.query,
        on=["subject_id", "hadm_id"],
    ),
    ops=diagnoses_ops,
).run()
print(f"{len(diagnoses)} rows extracted!")

2023-01-30 15:16:12,872 INFO cyclops.query.orm - Query returned successfully!
2023-01-30 15:16:12,873 INFO cyclops.utils.profile - Finished executing function run_query in 0.874558 s


263 rows extracted!


## Example 3. Advanced - uses `ConditionRegexMatch` from `cyclops.query.ops`. Get all patient encounters with diagnoses (ICD-9 long title contains `schizophrenia` and `chronic` ), in the year 2015.

In [4]:
admissions_ops = qo.Sequential(
    [
        qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
        qo.ConditionInYears("admittime", "2015"),
    ]
)
diagnoses_ops = qo.Sequential(
    [
        qo.ConditionEquals("icd_version", 9),
        qo.ConditionRegexMatch("long_title", r"(?=.*schizophrenia)(?=.*chronic)"),
    ]
)
patients = mimic.patients()
admissions = mimic.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
    ops=admissions_ops,
)
diagnoses = mimic.diagnoses(
    join=qo.JoinArgs(
        join_table=admissions.query,
        on=["subject_id", "hadm_id"],
    ),
    ops=diagnoses_ops,
).run()
print(f"{len(diagnoses)} rows extracted!")

2023-01-30 15:16:14,524 INFO cyclops.query.orm - Query returned successfully!
2023-01-30 15:16:14,525 INFO cyclops.utils.profile - Finished executing function run_query in 1.533797 s


82 rows extracted!


## Example 4. Get routine vital signs for patients from year 2015, limit to 100 rows.

In [5]:
admissions_ops = qo.Sequential(
    [
        qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
        qo.ConditionInYears("admittime", "2015"),
    ]
)
chartevents_ops = qo.Sequential([qo.ConditionEquals("category", "Routine Vital Signs")])
patients = mimic.patients()
admissions = mimic.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
    ops=admissions_ops,
)
vitals = mimic.chartevents(
    join=qo.JoinArgs(
        join_table=admissions.query,
        on=["subject_id", "hadm_id"],
    ),
    ops=chartevents_ops,
).run(limit=100)
print(f"{len(vitals)} rows extracted!")

2023-01-30 15:17:49,765 INFO cyclops.query.orm - Query returned successfully!
2023-01-30 15:17:49,765 INFO cyclops.utils.profile - Finished executing function run_query in 95.007937 s


100 rows extracted!


## Example 5. Get hemoglobin lab tests for patients from year 2009, limit to 100 rows.

In [6]:
admissions_ops = qo.Sequential(
    [
        qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
        qo.ConditionInYears("admittime", "2009"),
    ]
)
labevents_ops = qo.Sequential([qo.ConditionEquals("label", "hemoglobin")])
patients = mimic.patients()
admissions = mimic.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
    ops=admissions_ops,
)
labs = mimic.chartevents(
    join=qo.JoinArgs(
        join_table=admissions.query,
        on=["subject_id", "hadm_id"],
    ),
    ops=labevents_ops,
).run(limit=100)
print(f"{len(labs)} rows extracted!")

2023-01-30 15:19:03,562 INFO cyclops.query.orm - Query returned successfully!
2023-01-30 15:19:03,563 INFO cyclops.utils.profile - Finished executing function run_query in 73.678148 s


100 rows extracted!


## Example 6. Get all female patient encounters from year 2015, and return as dask dataframe (lazy evaluation) with 4 partitions (batches) aggregated based on `subject_id`.

In [7]:
admissions_ops = qo.Sequential(
    [
        qo.AddDeltaColumn(["admittime", "dischtime"], years="anchor_year_difference"),
        qo.ConditionInYears("admittime", "2015"),
        qo.Cast("gender", "str"),
        qo.ConditionEquals("gender", "F"),
    ]
)
patients = mimic.patients()
admissions = mimic.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
    ops=admissions_ops,
).run(backend="dask", index_col="subject_id", n_partitions=4)
print(f"{len(admissions)} rows extracted!")
print(f"Return type: {type(admissions)}")
print(f"Number of partitions: {admissions.npartitions}")

2023-01-30 15:19:04,056 INFO cyclops.query.orm - Query returned successfully!
2023-01-30 15:19:04,057 INFO cyclops.utils.profile - Finished executing function run_query in 0.442855 s


35639 rows extracted!
Return type: <class 'dask.dataframe.core.DataFrame'>
Number of partitions: 4


## Example 7. Running a raw SQL string.

In [8]:
data = mimic._db.run_query("SELECT * FROM mimiciv_hosp.admissions LIMIT 100")
print(f"{len(data)} rows extracted!")

2023-01-30 15:19:05,133 INFO cyclops.query.orm - Query returned successfully!
2023-01-30 15:19:05,134 INFO cyclops.utils.profile - Finished executing function run_query in 0.006734 s


100 rows extracted!
